In [ ]:
# Databricks notebook source
# MAGIC %md
# MAGIC # ETL Process for Customer 360 Data
# MAGIC This notebook performs an ETL process to create a comprehensive Customer 360 dataset using data from various Unity Catalog tables.

# COMMAND ----------

import pyspark.sql.functions as F
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# COMMAND ----------

# MAGIC %md
# MAGIC ## Step 1: Data Source Configuration
# MAGIC Load data from Unity Catalog tables.

# COMMAND ----------

try:
    logger.info("Loading data from Unity Catalog tables")
    policy_df = spark.table("catalog.source_db.policy")
    claims_df = spark.table("catalog.source_db.claims")
    demographics_df = spark.table("catalog.source_db.demographics")
    scores_df = spark.table("catalog.source_db.scores")
    aiml_insights_df = spark.table("catalog.source_db.aiml_insights")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Step 2: Data Selection and Filtering
# MAGIC Select relevant fields from each data source.

# COMMAND ----------

    logger.info("Selecting relevant fields from each data source")
    demographics_selected = demographics_df.select(
        "Customer_ID", "Customer_Name", "Email", "Phone_Number", "Address", "City", "State", "Postal_Code",
        "Date_of_Birth", "Gender", "Marital_Status", "Occupation", "Income_Level", "Customer_Segment"
    )
    claims_selected = claims_df.select(
        "Claim_ID", "Policy_ID", "Claim_Date", "Claim_Type", "Claim_Status", "Claim_Amount", "Claim_Payout"
    )
    policy_selected = policy_df.select(
        "policy_id", "customer_id", "policy_type", "policy_status", "policy_start_date", "policy_end_date",
        "policy_term", "policy_premium", "total_premium_paid", "renewal_status", "policy_addons"
    )

# COMMAND ----------

# MAGIC %md
# MAGIC ## Step 3: Data Integration
# MAGIC Join datasets based on key identifiers.

# COMMAND ----------

    logger.info("Joining datasets based on key identifiers")
    joined_df = demographics_selected.join(
        policy_selected, demographics_selected.Customer_ID == policy_selected.customer_id, "inner"
    ).join(
        claims_selected, policy_selected.policy_id == claims_selected.Policy_ID, "inner"
    )

    # Cache the joined DataFrame if reused multiple times
    joined_df.cache()

# COMMAND ----------

# MAGIC %md
# MAGIC ## Step 4: Data Aggregation
# MAGIC Compute aggregate metrics.

# COMMAND ----------

    logger.info("Computing aggregate metrics")
    aggregated_df = joined_df.groupBy("Customer_ID").agg(
        F.count("Claim_ID").alias("Total_Claims"),
        F.countDistinct("policy_id").alias("Policy_Count"),
        F.max("Claim_Date").alias("Recent_Claim_Date"),
        F.avg("Claim_Amount").alias("Average_Claim_Amount")
    )

# COMMAND ----------

# MAGIC %md
# MAGIC ## Step 5: Custom Calculations
# MAGIC Derive additional insights.

# COMMAND ----------

    logger.info("Deriving additional insights")
    final_df = aggregated_df.join(demographics_selected, "Customer_ID", "inner").withColumn(
        "Age", F.datediff(F.current_date(), F.to_date("Date_of_Birth")) / 365
    ).withColumn(
        "Claim_To_Premium_Ratio", F.when(F.col("total_premium_paid") != 0, F.col("Average_Claim_Amount") / F.col("total_premium_paid")).otherwise(0)
    ).withColumn(
        "Claims_Per_Policy", F.when(F.col("Policy_Count") != 0, F.col("Total_Claims") / F.col("Policy_Count")).otherwise(0)
    ).withColumn(
        "Retention_Rate", F.lit(0.85)
    ).withColumn(
        "Cross_Sell_Opportunities", F.lit("Multi-Policy Discount, Home Coverage Add-on")
    ).withColumn(
        "Upsell_Potential", F.lit("Premium Vehicle Coverage")
    )

# COMMAND ----------

# MAGIC %md
# MAGIC ## Step 6: Final Data Assembly
# MAGIC Combine all transformed data into a single dataset.

# COMMAND ----------

    logger.info("Combining all transformed data into a single dataset")
    customer_360_df = final_df.join(scores_df, "Customer_ID", "inner").join(aiml_insights_df, "Customer_ID", "inner")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Step 7: Output Data
# MAGIC Write the final dataset to a Unity Catalog table.

# COMMAND ----------

    logger.info("Writing the final dataset to a Unity Catalog table")
    customer_360_df.write.format("delta").mode("overwrite").saveAsTable("catalog.target_db.customer_360")

    logger.info("ETL process completed successfully")

except Exception as e:
    logger.error("An error occurred during the ETL process", exc_info=True)
